In [1]:
from Agents.utils import split_notebook_turns
from Agents.agents import turn_classifier
import json
from langchain_core.messages import FunctionMessage, HumanMessage

{"detail":"User is not allowed to create run. UserProjectRole with User Email 'daniel.oluwakayode-c@turing.com', Engagement Name 'public-engagement' and Project Name 'beta-test' not found"}
Authentication failed.


In [2]:
conversation_turns = split_notebook_turns("Data/python_basics_&_scripting__write_code_in_python__22__24_03_2024_08_57_45_10.ipynb")

In [8]:
conversation_turns[1]

[{'role': 'User',
  'content': "Right now, the code doesn't explicitly verify if the write operation was successful. Are there ways to check if the new name was written correctly to the file?"},
 {'role': 'Assistant',
  'content': "You're absolutely right, verifying the write operation's success is crucial for ensuring data integrity. Here are two ways to check if the new name was written correctly to the file in the provided Python script:\n\n1. `Using file.tell()`:\n\n* After writing the new name data using `file.write()`, you can call `file.tell()`. This method returns the current position of the file pointer."},
 {'role': 'Assistant',
  'content': '# After writing the new name...\nnew_name_end_position = file.tell()\n\n# Compare the written bytes with the expected size of the new name data\nexpected_bytes_written = len(new_name_encoded)\nif new_name_end_position - byte_offset == expected_bytes_written:\n    print("Employee record updated successfully!")\nelse:\n    print(\n        

In [7]:
result = conversation_turns[0]
result = turn_classifier([HumanMessage("hello how are you?")])
print(result)
all([result["has_code"], result["complete_code"], result["can_be_tested"]])

Validating output schema.....
{'has_code': False, 'complete_code': False, 'can_be_tested': False, 'code': '', 'dependencies': [], 'language': ''}


False

In [6]:
result = turn_classifier([HumanMessage("print('i am fine)")])
print(result)


Validating output schema.....
{'has_code': True, 'complete_code': False, 'can_be_tested': False, 'code': "print('i am fine)", 'dependencies': [], 'language': 'python3'}
